In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\vaibh\\OneDrive\\Desktop\\full_model\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\vaibh\\OneDrive\\Desktop\\full_model'

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name : str
    aplha : float
    l1_ratio : float
    target_column : str
    

In [16]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml , create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)    
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.Target_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            aplha=params.alpha, 
            l1_ratio=params.l1_ratio,
            target_column=schema.target_column

        )
        return model_trainer_config
        



In [18]:
import pandas as pd
from sklearn.linear_model import ElasticNet
import joblib
from src.mlProject import logger
import os



In [19]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):

        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop(columns=[self.config.target_column], axis=1)
        test_x = test_data.drop(columns=[self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        lr = ElasticNet(
            alpha=self.config.aplha,
            l1_ratio=self.config.l1_ratio
        )
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [20]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config= model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-06-12 12:10:41,644: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-12 12:10:41,644: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-12 12:10:41,656: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-12 12:10:41,658: INFO: common: created directory at: artifacts]


BoxKeyError: "'ConfigBox' object has no attribute 'target_column'"